In [ ]:
%%bash 
wget https://storage.googleapis.com/tunbert-opensource-datasets/TRCD_dataset/TRCD_train.json
wget https://storage.googleapis.com/tunbert-opensource-datasets/TRCD_dataset/TRCD_valid.json
wget https://storage.googleapis.com/tunbert-opensource-datasets/TRCD_dataset/TRCD_test.json

--2022-06-29 15:52:46--  https://storage.googleapis.com/tunbert-opensource-datasets/TRCD_dataset/TRCD_train.json
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.69.128, 173.194.195.128, 173.194.197.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 701430 (685K) [application/octet-stream]
Saving to: ‘TRCD_train.json’

     0K .......... .......... .......... .......... ..........  7% 49.9M 0s
    50K .......... .......... .......... .......... .......... 14% 58.5M 0s
   100K .......... .......... .......... .......... .......... 21% 69.1M 0s
   150K .......... .......... .......... .......... .......... 29% 54.4M 0s
   200K .......... .......... .......... .......... .......... 36% 68.7M 0s
   250K .......... .......... .......... .......... .......... 43% 86.1M 0s
   300K .......... .......... .......... .......... .......... 51% 84.5M 0s
   350K .......... .

In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.metrics import f1_score,accuracy_score,accuracy_score,recall_score

In [ ]:
train=pd.read_json("/content/TRCD_train.json")
valid=pd.read_json("/content/TRCD_valid.json")
test=pd.read_json("/content/TRCD_test.json")

In [ ]:
train

,data,version
0,{'paragraphs': [{'context': 'النائب في مجلس نو...,0.1
1,{'paragraphs': [{'context': 'صلاحيات رئيس الحك...,0.1
2,{'paragraphs': [{'context': 'صلاحيات رئيس الحك...,0.1
3,{'paragraphs': [{'context': 'الحكومة هي المسؤو...,0.1
4,{'paragraphs': [{'context': 'يلزم رئيس الجمهور...,0.1
...,...,...
109,{'paragraphs': [{'context': 'حرية الرأي والتفك...,0.1
110,{'paragraphs': [{'context': 'وكل تونسي وتونسية...,0.1
111,{'paragraphs': [{'context': 'الحق في الثقافة م...,0.1
112,{'paragraphs': [{'context': 'الدولة تشجع على ا...,0.1


In [ ]:
# !nvidia-smi
# !git clone https://github.com/huggingface/transformers \
# && cd transformers \
# !pip install ./transformers
# !pip install tensorboardX

Wed Jun 29 15:36:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
class cfg: 
  pretrained_model="UBC-NLP/MARBERTv2"
  input_shape=128
  learning_rate= 1e-4
  batch_size=128
  epochs=9

In [ ]:
def get_strategy():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experi3mental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        auto = tf.data.experimental.AUTOTUNE
        replicas = strategy.num_replicas_in_sync
        print(f'TPU: {tpu.master()}')
    except:
        strategy = tf.distribute.get_strategy()
        auto = tf.data.experimental.AUTOTUNE
        replicas = strategy.num_replicas_in_sync

    tf.config.optimizer.set_jit(True)
    print(f'Replicas: {replicas}')

    return strategy

strategy=get_strategy()

Replicas: 1


In [ ]:
def tokenize(sentences):
    input_ids, attention_mask = [],[]
    tokenizer=AutoTokenizer.from_pretrained(cfg.pretrained_model)
    for sentence in sentences:
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True,max_length=cfg.input_shape,truncation=True, padding='max_length',return_attention_mask=True, return_token_type_ids=True)
        input_ids.append(inputs['input_ids'])
        attention_mask.append(inputs['attention_mask'])
    return {"input_ids":tf.convert_to_tensor(input_ids),"attention_masks":tf.convert_to_tensor(attention_mask)}

In [ ]:
def create_model():
    with strategy.scope():
      config =AutoConfig.from_pretrained(cfg.pretrained_model,dropout=0.3,seed=3,attention_dropout=0.3,output_hidden_states = True)
      transformer= TFAutoModel.from_pretrained(cfg.pretrained_model)
      input_ids= tf.keras.layers.Input(shape=(cfg.input_shape,), dtype='int32',name="input_ids")
      input_masks = tf.keras.layers.Input(shape=(cfg.input_shape,), dtype='int32',name="attention_masks")
      embedding_layer=transformer(input_ids, attention_mask=input_masks)[0][:,0,:]
      output=tf.keras.layers.Dense(1, activation="sigmoid",name="output")(embedding_layer)
                                                  
      model = tf.keras.Model(inputs=[input_ids, input_masks], outputs = output)
      
    return model                                    

In [ ]:
!git clone https://huggingface.co/UBC-NLP/MARBERTv2

Cloning into 'MARBERTv2'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 27 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (27/27), done.


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.3 MB/s 
     |████████████████████████████████| 140 kB 67.5 MB/s 
     |████████████████████████████████| 1.1 MB 65.7 MB/s 
     |████████████████████████████████| 212 kB 75.6 MB/s 
     |████████████████████████████████| 127 kB 73.9 MB/s 
     |████████████████████████████████| 271 kB 74.5 MB/s 
     |████████████████████████████████| 144 kB 69.7 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import tensorflow
tensorflow.__version__

'2.8.2'

In [ ]:
!pip install git+https://github.com/huggingface/transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-yq7z0uum
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-yq7z0uum
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 59.7 MB/s 
     |████████████████████████████████| 101 kB 13.6 MB/s 
  Created wheel for transformers: filename=transformers-4.21.0.dev0-py3-none-any.whl size=4553383 sha256=1c8ae45b03c6404689754eb925196c02f6de12e7e2aa80339c7891c386493070
  Stored in directory: /tmp/pip-ephem-wheel-cache-55mlp_lf/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninst

In [ ]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 99980, done.
remote: Counting objects: 100% (492/492), done.
remote: Compressing objects: 100% (295/295), done.
remote: Total 99980 (delta 259), reused 307 (delta 165), pack-reused 99488
Receiving objects: 100% (99980/99980), 93.40 MiB | 25.80 MiB/s, done.
Resolving deltas: 100% (73610/73610), done.


In [ ]:
!cat /content/MARBERTv2/pytorch_model.bin

version https://git-lfs.github.com/spec/v1
oid sha256:5d34aca06d9ec233e8483478faec43cea7c819d1ea4f83fc99d9c8067dcf8bab
size 654190940


In [ ]:
!python /content/transformers/examples/pytorch/question-answering/run_qa.py \
--model_name_or_path /content/MARBERTv2 \
--output_dir output \
--train_file=/content/TRCD_train.json \
--validation_file=/content/TRCD_valid.json \
--max_seq_length 128 \
--do_train \
--do_eval \

06/29/2022 16:15:04 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
06/29/2022 16:15:04 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=N

In [ ]:
!git lfs install
!git clone https://huggingface.co/UBC-NLP/MARBERTv2

Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 'MARBERTv2'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 27 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (27/27), done.
Filtering content: 100% (2/2), 1.21 GiB | 46.68 MiB/s, done.


In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json 
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2022-06-29 15:59:04--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   207MB/s    in 0.2s    

2022-06-29 15:59:05 (207 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2022-06-29 15:59:05--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

In [ ]:
# def make_dataset(data):
#     inputs=tokenize(data.sentence.values.astype(str))
#     target=data.label
#     train = tf.data.Dataset.from_tensor_slices(((inputs["input_ids"],inputs["attention_masks"]),target)).batch(cfg.batch_size)
#     return train

# train_ds=make_dataset(train)
# valid_ds=make_dataset(valid)
# test_ds=make_dataset(test)